In [27]:
import pandas as pd
import numpy as np
# import datetime
from datetime import datetime, timezone
import requests
import json
import matplotlib.pyplot as plt
import math
import scipy.stats as st

In [28]:
def create_tuple(response):
    # all data
    response_data = response.json()
    # print(response_data)
    # data for a single target
    response_datum = response_data[0]
    # print(response_datum)
    # the target - e.g. "120MapleAve.STM.M22-V/AverageMassFlow"
    response_target = response_datum['target']
    # the metric for this target - e.g. "Steam"
    #response_metric = response_datum['metric']
    # print(response_metric)
    # the datapoints as a list of lists
    response_datapoints = response_datum['datapoints']
    # print(response_datapoints)
    # array of tuples for numpy
    tuple_array = [tuple(x) for x in response_datapoints]
    # make a numpy array
    npa = np.array(tuple_array, dtype=[
                   ('value', np.double), ('ts', 'datetime64[ms]')])
    # npa = npa.astype([('value', np.double), ('ts', datetime)])

    # print("here")
    # for index, i in enumerate(npa):
    #   j = pd.to_datetime(i[1]).replace(tzinfo=timezone.utc).astimezone(tz="America/New_York")
    #   npa[index][1] = j
    return npa

In [29]:
def current_date():
    return str(datetime.now())

In [30]:
"""
Returns an array of tuples where tuple[0] is the metric, and tuple[1] is the time 
of measurement

Target: a WebCTRL point name in the form of a string 
Example: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log"

Server: a string representing the WebCTRL server our point exists in
Example: "biotech_main"

Start: the start time we wish to be querying data about (in string format)
Example: "2021-12-31T11:00Z"

End: The end time we wish to be querying data about
Example: "2022-12-31T11:00Z"
"""
def fume_query(target,server, start,end):
  url = "https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query"
  data = {
      "range": {
        "from": start,
        "to": end,
      },
      "targets": [
        {
          "payload": {
            "schema": server,
          },
          "target": target
        }
      ],

    }
  request = requests.post(url, json=data)
  print(request)
  # print(request.json())
  return create_tuple(request)

In [31]:
"""
Returns a time series of the ithaca air temperature over the respective 
"""

def outside_temp(start,end):
  # https://ypsu0n34jc.execute-api.us-east-1.amazonaws.com/dev/query doesn't work
  # https://portal.emcs.cornell.edu/api/datasources/proxy/5/query works
  # https://portal-api.emcs.cucloud.net/query works
  url = "https://portal-api.emcs.cucloud.net/query"
  target = "GameFarmRoadWeatherStation.TAVG_H_F"
  data = {
    "range": {
        "from": start,
        "to": end
    },
    "targets": [
        {
            "target": target,
        }
    ]
  }
  request = requests.post(url, json=data)
  print(request)
#   print(request.json())
  return create_tuple(request)

In [38]:
from datetime import datetime
from dateutil import parser
import matplotlib.dates as mdates

def btuh(target_list, server, start, end):
  cfm_master = fume_query(target_list[0],server,start,end)
  sash_master = fume_query(target_list[3],server,start,end)
  occ_master = fume_query(target_list[2],server,start,end)
  internal_temp_master = fume_query(target_list[1],server,start,end)
  external_temp_master = outside_temp(start,end)
  print("\nExternal Temp Master ", external_temp_master)

  cfm_list = pd.Series(data=[i[0] for i in cfm_master], index=[i[1] for i in cfm_master])
  print("CFM List: ", cfm_list)
  cfm_list = cfm_list[~cfm_list.index.duplicated()]
  print("CFM List new: ", cfm_list)

  sash_list = pd.Series(data=[i[0] for i in sash_master], index=[i[1] for i in sash_master])
  print("\nSash List: ", sash_list)
  sash_list = sash_list[~sash_list.index.duplicated()]
  print("\nSash List new: ", sash_list)

  occ_list = pd.Series(data=[i[0] for i in occ_master], index=[i[1] for i in occ_master])
  print("\nOCC List: ", occ_list)
  occ_list = occ_list[~occ_list.index.duplicated()]
  print("\nOCC List new: ", occ_list)
  
  internal_temp_list = pd.Series(data=[i[0] for i in internal_temp_master], index=[i[1] for i in internal_temp_master])
  print("\nInternal Temp List: ", internal_temp_list)
  internal_temp_list = internal_temp_list[~internal_temp_list.index.duplicated()]
  print("\nInternal Temp List new: ", internal_temp_list)

  external_temp_list = pd.Series(data=[i[0] for i in external_temp_master], index=[i[1] for i in external_temp_master])
  print("\nExternal Temp List: ", external_temp_list)
  # external_temp_list = pd.read_csv("C:/Users/Dan/Documents/GitHub/fume-hood-dashboard/game_farm/hist.csv", index_col=0).squeeze()
  external_temp_list.index = external_temp_list.index.astype('datetime64[ns]')
  print("\nExternal Temp List: ", external_temp_list)
  
  # iterate over external_temp_list (pd.Series)
  # create new pd.Series by adding hourly measurement of external temp
  # then calculate: difference = (ext2 - ext1)/12
  # add 12 new values, each increasing by difference

  new_external_temp_list = pd.Series(data=[i[0] for i in internal_temp_master])
  print("\nNew External Temp List: ", new_external_temp_list)

  df = pd.concat([cfm_list, sash_list, occ_list, internal_temp_list, external_temp_list], axis=1)
  df.columns = ["cfm", "sash" ,"occ", "internal temp", "external temp"]
  display(df)

  df["BTUh"] = 1.08 * df["cfm"] * (df["internal temp"] - df["external temp"])

  display(df)

  df2 = pd.concat([cfm_list, sash_list, occ_list, internal_temp_list], axis=1)
  df2.columns = ["cfm", "sash" ,"occ", "internal temp"]
  df2["BTUh"] = df["BTUh"]
  display(df2)
  df2 = df2.dropna()
  display(df2)


In [33]:
target_list = ["#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hoodvalve_flow/trend_log",
               "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log",
               "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend",
               "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash"]
server = "biotech_main"
start = str(datetime(2022, 11, 17))
end = str(datetime(2022, 11, 18))

btuh(target_list,server,start,end)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
CFM List:  2022-11-17 05:00:00    224.13
2022-11-17 05:05:00    224.13
2022-11-17 05:10:00    224.13
2022-11-17 05:15:00    225.10
2022-11-17 05:20:00    225.10
                        ...  
2022-11-19 04:35:00    224.13
2022-11-19 04:40:00    225.59
2022-11-19 04:45:00    227.06
2022-11-19 04:50:00    227.06
2022-11-19 04:55:00    221.20
Length: 576, dtype: float64
CFM List new:  2022-11-17 05:00:00    224.13
2022-11-17 05:05:00    224.13
2022-11-17 05:10:00    224.13
2022-11-17 05:15:00    225.10
2022-11-17 05:20:00    225.10
                        ...  
2022-11-19 04:35:00    224.13
2022-11-19 04:40:00    225.59
2022-11-19 04:45:00    227.06
2022-11-19 04:50:00    227.06
2022-11-19 04:55:00    221.20
Length: 576, dtype: float64

Sash List:  2022-11-17 05:00:00    1.2
2022-11-17 05:05:00    1.2
2022-11-17 05:10:00    1.2
2022-11-17 05:15:00    1.2
2022-11-17 05:20:00    1.2
                      ...

,cfm,sash,occ,internal temp,external temp
2022-11-17 00:00:00,NaN,NaN,NaN,NaN,37.6286
2022-11-17 01:00:00,NaN,NaN,NaN,NaN,37.1696
2022-11-17 02:00:00,NaN,NaN,NaN,NaN,36.5900
2022-11-17 03:00:00,NaN,NaN,NaN,NaN,36.2030
2022-11-17 04:00:00,NaN,NaN,NaN,NaN,36.1958
...,...,...,...,...,...
2022-11-19 04:35:00,224.13,1.2,0.0,70.3,NaN
2022-11-19 04:40:00,225.59,1.2,0.0,70.3,NaN
2022-11-19 04:45:00,227.06,1.2,0.0,70.2,NaN
2022-11-19 04:50:00,227.06,1.2,0.0,70.2,NaN


,cfm,sash,occ,internal temp,external temp,BTUh
2022-11-17 00:00:00,NaN,NaN,NaN,NaN,37.6286,NaN
2022-11-17 01:00:00,NaN,NaN,NaN,NaN,37.1696,NaN
2022-11-17 02:00:00,NaN,NaN,NaN,NaN,36.5900,NaN
2022-11-17 03:00:00,NaN,NaN,NaN,NaN,36.2030,NaN
2022-11-17 04:00:00,NaN,NaN,NaN,NaN,36.1958,NaN
...,...,...,...,...,...,...
2022-11-19 04:35:00,224.13,1.2,0.0,70.3,NaN,NaN
2022-11-19 04:40:00,225.59,1.2,0.0,70.3,NaN,NaN
2022-11-19 04:45:00,227.06,1.2,0.0,70.2,NaN,NaN
2022-11-19 04:50:00,227.06,1.2,0.0,70.2,NaN,NaN


,cfm,sash,occ,internal temp,BTUh
2022-11-17 05:00:00,224.13,1.2,1.0,72.2,8768.347518
2022-11-17 05:05:00,224.13,1.2,1.0,72.4,NaN
2022-11-17 05:10:00,224.13,1.2,1.0,72.4,NaN
2022-11-17 05:15:00,225.10,1.2,1.0,72.1,NaN
2022-11-17 05:20:00,225.10,1.2,1.0,71.8,NaN
...,...,...,...,...,...
2022-11-19 04:35:00,224.13,1.2,0.0,70.3,NaN
2022-11-19 04:40:00,225.59,1.2,0.0,70.3,NaN
2022-11-19 04:45:00,227.06,1.2,0.0,70.2,NaN
2022-11-19 04:50:00,227.06,1.2,0.0,70.2,NaN


,cfm,sash,occ,internal temp,BTUh
2022-11-17 05:00:00,224.13,1.2,1.0,72.2,8768.347518
2022-11-17 06:00:00,222.66,1.2,0.0,72.2,8853.246605
2022-11-17 07:00:00,227.55,1.2,0.0,71.1,8913.153524
2022-11-17 08:00:00,224.13,1.2,0.0,70.7,8739.881214
2022-11-17 09:00:00,226.57,1.2,0.0,70.4,8854.114530
2022-11-17 10:00:00,220.71,1.2,0.0,70.4,8813.469410
2022-11-17 11:00:00,226.08,1.2,1.0,70.2,9241.014574
2022-11-17 12:00:00,222.17,1.2,0.0,71.2,9297.814500
2022-11-17 13:00:00,223.15,1.2,1.0,72.2,9409.778489
2022-11-17 14:00:00,223.15,1.2,1.0,72.0,9046.202872
